In [1]:
import random
import time
from math import sqrt
from functools import lru_cache
from collections import deque, defaultdict
from heapq import heappush, heappop

class Cell:
    def __init__(self, x, y):
        self.x, self.y, self.ent = x, y, None
    def empty(self): return self.ent is None

class Entity:
    def __init__(self, color): self.color = color

class Eater(Entity): pass

class Food(Entity):
    def __init__(self, color, is_last=False):
        super().__init__(color)
        self.is_last = is_last

class Grid:
    DIRS = [(0,1), (0,-1), (1,0), (-1,0)]
    COLOR_MAP = {
        'Food': {'White': 0, 'Green': 1, 'Blue': 2, 'Red': 3, 'Yellow': 4, 'Purple': 5, 'Pink': 6, 'Brown': 7},
        'Eater': {'Green': 0, 'Blue': 1, 'Red': 2, 'Yellow': 3, 'Purple': 4, 'Pink': 5, 'Brown': 6}
    }

    def __init__(self, rows, cols):
        self.rows, self.cols = rows, cols
        self.grid = [[Cell(x, y) for y in range(cols)] for x in range(rows)]
        self.moves = []
        self.min_separation = 2
        self._distance_cache = {}
        self.solution_paths = {}
        self.difficulty_score = 0

    def place(self, ent, x, y):
        self.grid[x][y].ent = ent

    def clear(self):
        for row in self.grid:
            for cell in row: cell.ent = None
        self.moves = []
        self._distance_cache = {}
        self.solution_paths = {}

    @lru_cache(maxsize=1024)
    def _manhattan_distance(self, pos1, pos2):
        return abs(pos1[0] - pos2[0]) + abs(pos1[1] - pos2[1])

    def analyze_board_complexity(self):
        self.difficulty_score = 0
        colors_present = self._get_colors_present()
        if not colors_present: return 0
        
        # Factor 1: Multiple solution paths
        self._calculate_solution_paths()
        path_scores = []
        for color in colors_present:
            paths = len(self.solution_paths.get(color, []))
            path_scores.append(min(paths, 5) * 20)  # Cap at 5 paths per color
        
        # Factor 2: Color interaction complexity
        blocking_score = self._calculate_blocking_complexity()
        
        # Factor 3: Decision points (junctions)
        junction_score = self._calculate_junction_complexity()
        
        # Factor 4: Path interleaving
        interleaving_score = self._calculate_path_interleaving()
        
        # Factor 5: Deceptive paths (paths that look correct but aren't)
        deceptive_score = self._calculate_deceptive_paths()
        
        self.difficulty_score = (sum(path_scores) / len(path_scores) if path_scores else 0) + \
                               blocking_score + junction_score + interleaving_score + deceptive_score
        
        return self.difficulty_score

    def _get_colors_present(self):
        colors = set()
        for row in self.grid:
            for cell in row:
                if isinstance(cell.ent, Eater):
                    colors.add(cell.ent.color)
        return colors

    def _calculate_solution_paths(self):
        self.solution_paths = {}
        for x in range(self.rows):
            for y in range(self.cols):
                if isinstance(self.grid[x][y].ent, Eater):
                    color = self.grid[x][y].ent.color
                    paths = self._find_all_valid_paths(x, y, color)
                    self.solution_paths[color] = paths
        return self.solution_paths

    def _find_all_valid_paths(self, eater_x, eater_y, color, max_paths=10):
        # Find the last food item of this color
        last_food = None
        for x in range(self.rows):
            for y in range(self.cols):
                if (isinstance(self.grid[x][y].ent, Food) and 
                    self.grid[x][y].ent.color == color and 
                    self.grid[x][y].ent.is_last):
                    last_food = (x, y)
                    break
            if last_food: break
        
        if not last_food: return []
        
        # Find all food of this color
        color_food = []
        for x in range(self.rows):
            for y in range(self.cols):
                if (isinstance(self.grid[x][y].ent, Food) and 
                    self.grid[x][y].ent.color == color):
                    color_food.append((x, y))
        
        paths = []
        queue = [(last_food, [last_food], set([last_food]))]
        while queue and len(paths) < max_paths:
            (curr_x, curr_y), path, visited = queue.pop(0)
            
            if (curr_x, curr_y) == (eater_x, eater_y) and set(color_food).issubset(visited):
                paths.append(path)
                continue
                
            for dx, dy in self.DIRS:
                nx, ny = curr_x + dx, curr_y + dy
                while self.in_bounds(nx, ny) and self._can_move_through(nx, ny, color):
                    if (nx, ny) not in visited:
                        new_path = path + [(nx, ny)]
                        new_visited = visited.copy()
                        new_visited.add((nx, ny))
                        queue.append(((nx, ny), new_path, new_visited))
                    nx, ny = nx + dx, ny + dy
                
        return paths

    def _can_move_through(self, x, y, color):
        cell = self.grid[x][y]
        if cell.empty(): 
            return True
        if isinstance(cell.ent, Food) and (cell.ent.color == color or cell.ent.color == 'White'):
            return True
        if isinstance(cell.ent, Eater) and cell.ent.color == color:
            return True
        return False

    def _calculate_blocking_complexity(self):
        colors = self._get_colors_present()
        if len(colors) <= 1: return 0
        
        blocking_pairs = 0
        for c1 in colors:
            for c2 in colors:
                if c1 != c2:
                    if self._colors_block_each_other(c1, c2):
                        blocking_pairs += 1
        
        return min(blocking_pairs * 25, 100)  # Cap at 100

    def _colors_block_each_other(self, color1, color2):
        # Check if food/eater of color1 blocks paths of color2
        for x in range(self.rows):
            for y in range(self.cols):
                if isinstance(self.grid[x][y].ent, (Food, Eater)) and self.grid[x][y].ent.color == color1:
                    if self._is_blocking_position(x, y, color2):
                        return True
        return False

    def _is_blocking_position(self, x, y, color):
        # Is this position blocking a path for the given color?
        paths = self.solution_paths.get(color, [])
        for path in paths:
            for px, py in path:
                if (px, py) == (x, y):
                    # Check if there's an alternative path that doesn't include this position
                    if not any(all((px, py) != pos for pos in alt_path) for alt_path in paths):
                        return True
        return False

    def _calculate_junction_complexity(self):
        junctions = []
        for x in range(self.rows):
            for y in range(self.cols):
                if self._is_junction(x, y):
                    junctions.append((x, y))
        
        return min(len(junctions) * 10, 100)

    def _is_junction(self, x, y):
        # A junction is a position where multiple paths diverge
        neighboring_empty = 0
        for dx, dy in self.DIRS:
            nx, ny = x + dx, y + dy
            if self.in_bounds(nx, ny) and self.grid[nx][ny].empty():
                neighboring_empty += 1
        return neighboring_empty > 2

    def _calculate_path_interleaving(self):
        colors = self._get_colors_present()
        if len(colors) <= 1: return 0
        
        interleaving_score = 0
        color_positions = {}
        
        for color in colors:
            positions = []
            for x in range(self.rows):
                for y in range(self.cols):
                    if isinstance(self.grid[x][y].ent, (Food, Eater)) and self.grid[x][y].ent.color == color:
                        positions.append((x, y))
            color_positions[color] = positions
        
        for c1, pos1 in color_positions.items():
            for c2, pos2 in color_positions.items():
                if c1 != c2:
                    # Calculate how interleaved the positions are
                    interleaving = 0
                    for p1 in pos1:
                        near_other = any(self._manhattan_distance(p1, p2) <= 2 for p2 in pos2)
                        if near_other:
                            interleaving += 1
                    
                    interleaving_score += (interleaving / max(len(pos1), 1)) * 20
        
        return min(interleaving_score, 100)

    def _calculate_deceptive_paths(self):
        # Deceptive paths look correct but lead to dead ends
        deceptive_score = 0
        for color, paths in self.solution_paths.items():
            if not paths: continue
            
            valid_path_positions = set()
            for path in paths:
                for pos in path:
                    valid_path_positions.add(pos)
            
            # Find dead end paths
            dead_ends = 0
            for x in range(self.rows):
                for y in range(self.cols):
                    if self.grid[x][y].empty() or (isinstance(self.grid[x][y].ent, Food) and 
                                                  self.grid[x][y].ent.color in (color, 'White')):
                        if (x, y) not in valid_path_positions and self._leads_to_dead_end(x, y, color):
                            dead_ends += 1
            
            deceptive_score += min(dead_ends * 15, 100)
        
        return min(deceptive_score, 100)

    def _leads_to_dead_end(self, x, y, color):
        # Check if this position leads to a dead end for the given color
        visited = set([(x, y)])
        queue = [(x, y)]
        found_exit = False
        
        while queue and not found_exit:
            curr_x, curr_y = queue.pop(0)
            
            for dx, dy in self.DIRS:
                nx, ny = curr_x + dx, curr_y + dy
                while self.in_bounds(nx, ny) and self._can_move_through(nx, ny, color):
                    if (nx, ny) not in visited:
                        visited.add((nx, ny))
                        queue.append((nx, ny))
                        
                        # Check if this is a valid part of a solution path
                        if any((nx, ny) in path for path in self.solution_paths.get(color, [])):
                            found_exit = True
                            break
                    
                    nx, ny = nx + dx, ny + dy
                
                if found_exit: break
                
        return not found_exit and len(visited) > 3  # Only count substantial dead ends

    def gen_puzzle(self, eaters, min_food, target_difficulty=None):
        best_board = None
        best_score_diff = float('inf')
        attempts = 0
        max_attempts = 20  # Limit attempts for performance
        
        while attempts < max_attempts:
            self.clear()
            eater_pos = []
            
            # Place eaters randomly
            colors = [c for c, n in eaters.items() for _ in range(n)]
            random.shuffle(colors)
            for color in colors:
                pos = self.get_empty()
                if not pos: break
                x, y = pos
                e = Eater(color)
                self.place(e, x, y)
                eater_pos.append({'e': e, 'x': x, 'y': y, 'count': 0})

            # Generate food paths
            active_eaters = eater_pos.copy()
            while active_eaters:
                moved_any = False
                for einfo in active_eaters[:]:
                    if einfo['count'] >= min_food:
                        active_eaters.remove(einfo)
                        continue
                    if self.move_eater(einfo):
                        moved_any = True
                    else:
                        active_eaters.remove(einfo)
                if not moved_any: break

            # Fill remaining spaces strategically to create challenging paths
            self._strategic_fill(eater_pos)
            
            # Calculate board difficulty
            difficulty = self.analyze_board_complexity()
            
            # If targeting a specific difficulty
            if target_difficulty is not None:
                score_diff = abs(difficulty - target_difficulty)
                if score_diff < best_score_diff:
                    best_score_diff = score_diff
                    best_board = self._clone_board()
                    
                    # If we're close enough to target, stop early
                    if score_diff < 50:
                        break
            elif best_board is None or difficulty > self.difficulty_score:
                best_board = self._clone_board()
            
            attempts += 1
        
        # Restore best board
        if best_board:
            self._restore_board(best_board)
        
        return self.difficulty_score

    def _clone_board(self):
        # Save current board state
        board_state = []
        for x in range(self.rows):
            row = []
            for y in range(self.cols):
                cell = self.grid[x][y]
                if cell.empty():
                    row.append(None)
                elif isinstance(cell.ent, Eater):
                    row.append(('Eater', cell.ent.color))
                elif isinstance(cell.ent, Food):
                    row.append(('Food', cell.ent.color, cell.ent.is_last))
            board_state.append(row)
        return {
            'board': board_state,
            'moves': self.moves.copy(),
            'difficulty': self.difficulty_score
        }

    def _restore_board(self, board_data):
        self.clear()
        for x in range(self.rows):
            for y in range(self.cols):
                item = board_data['board'][x][y]
                if item:
                    if item[0] == 'Eater':
                        self.place(Eater(item[1]), x, y)
                    elif item[0] == 'Food':
                        self.place(Food(item[1], is_last=item[2]), x, y)
        self.moves = board_data['moves'].copy()
        self.difficulty_score = board_data['difficulty']

    def _strategic_fill(self, eater_pos):
        # Strategically fill remaining spaces to create challenging paths
        remaining_spaces = self.count_empty()
        if remaining_spaces == 0: return
        
        eater_colors = {e['e'].color for e in eater_pos}
        if not eater_colors: return
        
        # Create some strategic food placements to increase complexity
        filled = 0
        color_cycle = list(eater_colors)
        random.shuffle(color_cycle)
        color_index = 0
        
        while filled < remaining_spaces and filled < len(color_cycle) * 3:  # Limit strategic placements
            color = color_cycle[color_index % len(color_cycle)]
            color_index += 1
            
            # Find a strategic position for this color
            strategic_pos = self._find_strategic_position(color)
            if strategic_pos:
                x, y = strategic_pos
                self.place(Food(color), x, y)
                filled += 1
        
        # Fill remaining spaces with random food
        empty_cells = [(x, y) for x in range(self.rows) for y in range(self.cols) 
                      if self.grid[x][y].empty()]
        random.shuffle(empty_cells)
        
        for x, y in empty_cells:
            color = random.choice(list(eater_colors))
            self.place(Food(color), x, y)

    def _find_strategic_position(self, color):
        # Find a position that would create interesting paths
        candidates = []
        
        # Look for positions that create junctions
        for x in range(self.rows):
            for y in range(self.cols):
                if not self.grid[x][y].empty(): continue
                
                # Check if this would create a junction
                neighboring_empty = sum(1 for dx, dy in self.DIRS 
                                       if self.in_bounds(x+dx, y+dy) and self.grid[x+dx][y+dy].empty())
                
                if neighboring_empty >= 2:
                    # Check if near other colors (increases interleaving)
                    near_other_colors = False
                    for dx, dy in self.DIRS:
                        nx, ny = x+dx, y+dy
                        if self.in_bounds(nx, ny) and not self.grid[nx][ny].empty():
                            ent = self.grid[nx][ny].ent
                            if isinstance(ent, (Food, Eater)) and ent.color != color:
                                near_other_colors = True
                                break
                    
                    score = neighboring_empty * 2
                    if near_other_colors:
                        score += 4
                    
                    candidates.append((score, x, y))
        
        if candidates:
            candidates.sort(reverse=True)  # Higher scores first
            return candidates[0][1], candidates[0][2]
        
        # If no good candidates, find any empty space
        empty = self.get_empty()
        return empty

    def move_eater(self, einfo):
        random.shuffle(self.DIRS)
        
        for dx, dy in self.DIRS:
            x, y = einfo['x'], einfo['y']
            new_x, new_y = self.get_max_pos(x, y, dx, dy)
            
            if new_x is None or (new_x == x and new_y == y):
                continue
            
            # Place food where eater was
            is_first_food = einfo['count'] == 0
            self.grid[x][y].ent = None
            self.place(einfo['e'], new_x, new_y)
            self.place(Food(einfo['e'].color, is_last=is_first_food), x, y)
            einfo['count'] += abs(new_x - x) + abs(new_y - y)
            
            self.moves.append({
                'eater_color': einfo['e'].color,
                'start': (new_x, new_y),
                'end': (x, y),
                'food_color': einfo['e'].color,
                'dist': abs(new_x - x) + abs(new_y - y)
            })
            
            einfo['x'], einfo['y'] = new_x, new_y
            return True
            
        return False

    def get_max_pos(self, x, y, dx, dy):
        nx, ny = x, y
        while self.in_bounds(nx + dx, ny + dy) and self.grid[nx + dx][ny + dy].empty():
            nx += dx
            ny += dy
        return (nx, ny) if (nx, ny) != (x, y) else (None, None)

    def get_empty(self):
        empty = [(x,y) for x in range(self.rows) for y in range(self.cols) 
                if self.grid[x][y].empty()]
        return random.choice(empty) if empty else None

    def in_bounds(self, x, y):
        return 0 <= x < self.rows and 0 <= y < self.cols

    def count_empty(self):
        return sum(1 for row in self.grid for cell in row if cell.empty())

    def to_scene(self, level_id):
        width, height = 1440, 2560
        step = max(180, width // self.cols - 24)
        
        start_x = width/2 - step*self.cols/2 + step*0.5
        start_y = height/2 - step*self.rows/2 + step*0.5 - 40

        scene = f"""[gd_scene load_steps=4 format=3 uid="{level_id}"]

[ext_resource type="Script" path="res://Levels/Level.cs" id="lvlid"]
[ext_resource type="PackedScene" uid="uid://byatslmwbvorg" path="res://Entities/Food/Food.tscn" id="foodid"]
[ext_resource type="PackedScene" uid="uid://dd570jgysudow" path="res://Entities/Eater/Eater.tscn" id="eaterid"]

[node name="Level" type="Node"]
script = ExtResource("lvlid")
{self.solution_metadata()}

[node name="Food" type="Node" parent="."]

[node name="Eaters" type="Node" parent="."]
"""
        for x in range(self.rows):
            for y in range(self.cols):
                cell = self.grid[x][y]
                pos_x = start_x + y*step
                pos_y = start_y + x*step
                
                if isinstance(cell.ent, Food):
                    scene += f"""
[node name="Food{x}_{y}" parent="Food" instance=ExtResource("foodid")]
position = Vector2({pos_x}, {pos_y})
BoardStatePositionId = Vector2i({x}, {y})
FoodType = {self.COLOR_MAP['Food'][cell.ent.color]}
IsLast = {"true" if cell.ent.is_last else "false"}
"""
                elif isinstance(cell.ent, Eater):
                    scene += f"""
[node name="Eater{x}_{y}" parent="Eaters" instance=ExtResource("eaterid")]
position = Vector2({pos_x}, {pos_y})
EaterType = {self.COLOR_MAP['Eater'][cell.ent.color]}
BoardStatePositionId = Vector2i({x}, {y})
ValidFoodTypes = [0, {self.COLOR_MAP['Food'][cell.ent.color]}]
"""
        return scene

    def solution_metadata(self):
        metadata = "metadata/solution = PackedVector2Array("
        for move in self.moves:
                metadata += f"{move['start'][0]}, {move['start'][1]}, {move['end'][0]}, {move['end'][1]}, "
        metadata = metadata[:-2] + ")" if self.moves else metadata + ")"
        return metadata

def gen_boards(num_select, total_gen, rows, cols, eaters, min_food, difficulty_range=None):
    boards = []
    attempts = 0
    max_tries = total_gen * 5
    
    # Define difficulty ranges (low, medium, high)
    difficulty_targets = None
    if difficulty_range:
        if difficulty_range == "easy":
            difficulty_targets = [100, 200]
        elif difficulty_range == "medium":
            difficulty_targets = [300, 400]
        elif difficulty_range == "hard":
            difficulty_targets = [500, 600]
    
    while len(boards) < total_gen and attempts < max_tries:
        g = Grid(rows, cols)
        target = None
        if difficulty_targets:
            target = random.randint(difficulty_targets[0], difficulty_targets[1])
        
        difficulty = g.gen_puzzle(eaters, min_food, target)
        
        empty_count = g.count_empty()
        if empty_count == 0:
            # All spaces filled, consider it a valid board
            boards.append({
                'board': g,
                'difficulty': difficulty
            })
            print(f"Generated board {len(boards)}/{total_gen} with difficulty {difficulty}")
        
        attempts += 1

    # Sort by difficulty (ascending or descending based on preference)
    boards.sort(key=lambda x: x['difficulty'], reverse=True)  # Higher difficulty first
    return boards[:num_select]

In [4]:
# Example usage
if __name__ == "__main__":
    # Configuration
    DIFFICULTY = {
        'tutorial': ({'Green': 1}, 3, 3, 2, 30, 1, 0),
        'very_easy': ({'Blue': 1, 'Green': 1}, 4, 4, 3, 30, 2, 0),
        'very_easy2': ({'Blue': 1, 'Green': 1}, 5, 4, 10, 30, 2, 1),
        'easy': ({'Blue': 1, 'Green': 1, 'Red': 1}, 5, 5, 15, 30, 3, 2),
        'easy2': ({'Blue': 1, 'Green': 1, 'Red': 1}, 6, 5, 15, 30, 3, 3),
        'medium': ({'Blue': 1, 'Green': 1, 'Red': 1, 'Yellow': 1}, 6, 6, 20, 20, 3, 4),
        'medium2': ({'Blue': 1, 'Green': 1, 'Red': 1, 'Yellow': 1}, 7, 6, 20, 20, 3, 4),
        'hard': ({'Blue': 1, 'Green': 1, 'Red': 1, 'Yellow': 1, 'Pink': 1}, 7, 7, 20, 20, 3, 5),
        'hard2': ({'Blue': 1, 'Green': 1, 'Red': 1, 'Yellow': 1, 'Pink': 1}, 8, 7, 20, 20, 4, 5),
        'veryhard': ({'Blue': 1, 'Green': 1, 'Red': 1, 'Yellow': 1, 'Pink': 1, 'Purple': 1}, 9, 7, 20, 20, 4, 6),
        'veryhard2': ({'Blue': 1, 'Green': 1, 'Red': 1, 'Yellow': 1, 'Pink': 1, 'Purple': 1}, 10, 7, 20, 20, 4, 6),
        'max': ({'Blue': 1, 'Green': 1, 'Red': 1, 'Yellow': 1, 'Pink': 1, 'Purple': 1, 'Brown': 1}, 11, 7, 20, 15, 4, 7),
        'max2': ({'Blue': 1, 'Green': 1, 'Red': 1, 'Yellow': 1, 'Pink': 1, 'Purple': 1, 'Brown': 1}, 11, 7, 20, 10, 5, 7)
    }

    lvlid = 1
    ts = time.time()
    print(f"[{ts}] Starting")
    for diff, (eaters, rows, cols, num_select, white_pct, min_food, minimum_qualified_colors) in DIFFICULTY.items():
        new_boards = gen_boards(num_select, 5000, rows, cols, eaters, min_food, "hard")
        ts = time.time()
        for board in new_boards:
            with open(f"../Levels/NewLevel{lvlid}.tscn", "w") as f:
                f.write(board["board"].to_scene(lvlid))
            lvlid += 1

[1741025095.5715697] Starting
Generated board 1/5000 with difficulty 100.0
Generated board 2/5000 with difficulty 100.0
Generated board 3/5000 with difficulty 100.0
Generated board 4/5000 with difficulty 100.0
Generated board 5/5000 with difficulty 100.0
Generated board 6/5000 with difficulty 100.0
Generated board 7/5000 with difficulty 100.0
Generated board 8/5000 with difficulty 100.0
Generated board 9/5000 with difficulty 100.0
Generated board 10/5000 with difficulty 100.0
Generated board 11/5000 with difficulty 100.0
Generated board 12/5000 with difficulty 100.0
Generated board 13/5000 with difficulty 100.0
Generated board 14/5000 with difficulty 100.0
Generated board 15/5000 with difficulty 100.0
Generated board 16/5000 with difficulty 100.0
Generated board 17/5000 with difficulty 100.0
Generated board 18/5000 with difficulty 100.0
Generated board 19/5000 with difficulty 100.0
Generated board 20/5000 with difficulty 100.0
Generated board 21/5000 with difficulty 100.0
Generated boa

KeyboardInterrupt: 